### Team 5 Members:
1. Abhishek Namdev Sawant
2. Erdenetuya Namsrai
3. Garrett Ringler
4. Sai Kishore Chintala
5. Vivek Reddy Karra


### GUI Code

In [1]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tkinter as tk
from tkinter import ttk, simpledialog, messagebox
import mysql.connector
from mysql.connector import Error

def fetch_appointments_data(start_date, end_date):
    """Fetch appointment data from MySQL database within a specified date range."""
    try:
        cnx = mysql.connector.connect(user='mm_team05_01', password='mm_team05_01Pass-', host='CSSQL', database='mm_team05_01')
        cursor = cnx.cursor()
        
        query = """
        SELECT  
        p.patient_id AS 'Patient Id', 
        p.patient_first_name AS 'Patient First Name', 
        p.patient_last_name AS 'Patient Last Name', 
        p.dateofbirth AS 'Date of Birth', 
        ill.description AS 'Illness Description' 
        FROM Patients AS p 
        JOIN Patients_has_Illness_Details AS pd ON p.patient_id = pd.Patients_patient_id 
        JOIN Illness_Details AS ill ON ill.illness_id = pd.Illness_details_illness_id 
        WHERE dateofbirth BETWEEN %s AND %s 
        ORDER BY dateofbirth;
        """
        
        cursor.execute(query, (start_date, end_date))
        result = cursor.fetchall()
        
        headers = [i[0] for i in cursor.description]
        
    except Error as e:
        print("Error while connecting to MySQL", e)
        return [], []  
    finally:
        if cursor:
            cursor.close()
        if cnx:
            cnx.close()
    
    return headers, result

def fetch_appointment_details(start_date, end_date):
    """Fetch detailed appointment data from MySQL database within a user-specified date range."""
    try:
        cnx = mysql.connector.connect(user='mm_team05_01', password='mm_team05_01Pass-', host='CSSQL', database='mm_team05_01')
        cursor = cnx.cursor()
        query = """
        SELECT  
        p.patient_id AS 'Patient ID', 
        p.patient_first_name AS 'Patient First Name', 
        p.patient_last_name AS 'Patient Last Name', 
        a.date AS 'Appointment Date', 
        a.time AS 'Appointment Time', 
        s.staff_id AS 'Staff Id', 
        s.staff_first_name AS 'Staff First Name', 
        s.staff_last_name AS 'Staff Last Name' 
        FROM Patients AS p 
        JOIN Appointment AS a ON p.patient_id = a.Patients_patient_id 
        JOIN Patients_has_Staff AS ps ON p.patient_id = ps.Patients_patient_id 
        JOIN Staff AS s ON s.staff_id = ps.Staff_staff_id 
        WHERE a.date BETWEEN %s AND %s 
        ORDER BY a.date;
        """
        cursor.execute(query, (start_date, end_date))
        result = cursor.fetchall()
        headers = [i[0] for i in cursor.description]
    except Error as e:
        print("Error while connecting to MySQL", e)
        return [], []  # Return empty lists on error
    finally:
        if cursor:
            cursor.close()
        if cnx:
            cnx.close()
    return headers, result

def fetch_patient_appointment_counts(start_date, end_date):
    """Fetch the number of appointments for each patient within a user-specified date range."""
    try:
        cnx = mysql.connector.connect(user='mm_team05_01', password='mm_team05_01Pass-', host='CSSQL', database='mm_team05_01')
        cursor = cnx.cursor()
        query = """
        SELECT  
        p.patient_id AS 'Patient Id', 
        p.patient_first_name AS 'Patient First Name', 
        p.patient_last_name AS 'Patient Last Name', 
        COUNT(a.date) AS 'Appointment Count' 
        FROM Patients AS p 
        JOIN Appointment AS a ON p.patient_id = a.patients_patient_id 
        WHERE a.date BETWEEN %s AND %s 
        GROUP BY p.patient_id 
        ORDER BY COUNT(a.date) DESC;
        """
        cursor.execute(query, (start_date, end_date))
        result = cursor.fetchall()
        headers = [i[0] for i in cursor.description]
    except Error as e:
        print("Error while connecting to MySQL", e)
        return [], []  # Return empty lists on error
    finally:
        if cursor:
            cursor.close()
        if cnx:
            cnx.close()
    return headers, result

def fetch_billing_information():
    """Fetch billing information including patient and staff details."""
    try:
        cnx = mysql.connector.connect(user='mm_team05_01', password='mm_team05_01Pass-', host='CSSQL', database='mm_team05_01')
        cursor = cnx.cursor()
        query = """
        SELECT  
        p.patient_id AS 'Patient Id', 
        CONCAT(p.patient_first_name, ' ', p.patient_last_name) AS 'Patient Name', 
        mr.record_id AS 'Medical Record Id', 
        b.max_paid AS 'Maximum Amount Paid', 
        phs.Staff_staff_id AS 'Staff Id', 
        CONCAT(s.staff_first_name, ' ', s.staff_last_name) AS 'Staff Name',
        CONCAT(mr.addmission_date,' - ', mr.discharge_date) AS 'Date of Service'
        FROM 
        Patients AS p 
        JOIN 
        (SELECT  
        Patients_patient_id, MAX(amount) AS max_paid 
        FROM 
        Billing AS b 
        GROUP BY Patients_patient_id 
        ORDER BY max_paid DESC 
        LIMIT 1) AS b ON p.patient_id = b.Patients_patient_id 
        JOIN Medical_Records AS mr ON p.patient_id = mr.Patients_patient_id 
        JOIN Patients_has_Staff AS phs ON p.patient_id = phs.Patients_patient_id 
        JOIN Staff AS s ON phs.Staff_staff_id = s.staff_id 
        JOIN Staff_Type AS st ON s.staff_id = st.Staff_staff_id; 
        """
        cursor.execute(query)
        result = cursor.fetchall()
        headers = [i[0] for i in cursor.description]
    except Error as e:
        print("Error while connecting to MySQL", e)
        return [], []  # Return empty lists on error
    finally:
        if cursor:
            cursor.close()
        if cnx:
            cnx.close()
    return headers, result

def fetch_unpaid_billing_records(payment_status):
    """Fetch billing records based on payment status, including patient details."""
    try:
        cnx = mysql.connector.connect(user='mm_team05_01', password='mm_team05_01Pass-', host='CSSQL', database='mm_team05_01')
        cursor = cnx.cursor()
        query = """
        SELECT  
        p.patient_id AS 'Patient Id', 
        CONCAT(p.patient_first_name, ' ', p.patient_last_name) AS 'Patient Name', 
        b.amount AS 'Amount', 
        b.date AS 'Billing Date' 
        FROM Patients AS p 
        JOIN Billing AS b ON p.patient_id = b.Patients_patient_id 
        WHERE b.payment_status = %s
        ORDER BY b.date;
        """
        cursor.execute(query, (payment_status,))
        result = cursor.fetchall()
        headers = [i[0] for i in cursor.description]
    except Error as e:
        print("Error while connecting to MySQL", e)
        return [], []  # Return empty lists on error
    finally:
        if cursor:
            cursor.close()
        if cnx:
            cnx.close()
    return headers, result

def fetch_avg_billing_amount_by_illness(start_date, end_date):
    """Fetch the average billing amount for each illness description within a specified date range."""
    try:
        cnx = mysql.connector.connect(user='mm_team05_01', password='mm_team05_01Pass-', host='CSSQL', database='mm_team05_01')
        cursor = cnx.cursor()
        query = """
        SELECT  
        il.description AS 'Illness Description', 
        ROUND(AVG(b.amount), 2) AS 'Average Billing Amount' 
        FROM Illness_Details AS il 
        JOIN Patients_has_Illness_Details AS pid ON il.illness_id = pid.Illness_Details_illness_id 
        JOIN Billing AS b ON pid.Patients_patient_id = b.Patients_patient_id 
        WHERE b.date BETWEEN %s AND %s
        GROUP BY il.description;
        """
        cursor.execute(query, (start_date, end_date))
        result = cursor.fetchall()
        headers = [i[0] for i in cursor.description]
    except Error as e:
        print("Error while connecting to MySQL", e)
        return [], []  # Return empty lists on error
    finally:
        if cursor:
            cursor.close()
        if cnx:
            cnx.close()
    return headers, result

def display_table(headers, records, title="Database Query Results"):
    """Display fetched data in a GUI table."""
    root = tk.Tk()
    root.title(title)

    tree = ttk.Treeview(root, columns=headers, show='headings')

    # Define the columns
    for header in headers:
        tree.heading(header, text=header)

    # Insert data into the treeview
    for row in records:
        tree.insert('', tk.END, values=row)

    tree.pack(side='left', fill='both', expand=True)

    # Add a scrollbar
    scrollbar = ttk.Scrollbar(root, orient=tk.VERTICAL, command=tree.yview)
    tree.configure(yscroll=scrollbar.set)
    scrollbar.pack(side='right', fill='y')

    root.mainloop()

def fetch_appointments_gui():
    """GUI to fetch appointments data directly."""
    start_date = simpledialog.askstring("Input", "Enter appointment start date (YYYY-MM-DD):")
    end_date = simpledialog.askstring("Input", "Enter appointment end date (YYYY-MM-DD):")
    if start_date and end_date:
        headers, records = fetch_appointments_data(start_date, end_date)
        if headers and records:
            display_table(headers, records, title=f"Appointments from {start_date} to {end_date}")
        else:
            tk.messagebox.showinfo("No Data", "No data found for the specified date range.")
    else:
        tk.messagebox.showinfo("Input Error", "Please enter valid start and end dates.")


def fetch_appointment_details_gui():
    """GUI to prompt user for date range and fetch detailed appointment data accordingly."""
    start_date = simpledialog.askstring("Input", "Enter appointment start date (YYYY-MM-DD):")
    end_date = simpledialog.askstring("Input", "Enter appointment end date (YYYY-MM-DD):")
    if start_date and end_date:
        headers, records = fetch_appointment_details(start_date, end_date)
        if headers and records:
            display_table(headers, records, title=f"Appointment Details from {start_date} to {end_date}")
        else:
            tk.messagebox.showinfo("No Data", "No appointment details found for the specified date range.")
    else:
        tk.messagebox.showinfo("Input Error", "Please enter valid start and end dates.")

def fetch_patient_appointment_counts_gui():
    """GUI to prompt user for date range and fetch the appointment count for each patient accordingly."""
    start_date = simpledialog.askstring("Input", "Enter appointment start date (YYYY-MM-DD):")
    end_date = simpledialog.askstring("Input", "Enter appointment end date (YYYY-MM-DD):")
    if start_date and end_date:
        headers, records = fetch_patient_appointment_counts(start_date, end_date)
        if headers and records:
            display_table(headers, records, title=f"Patient Appointment Counts from {start_date} to {end_date}")
        else:
            tk.messagebox.showinfo("No Data", "No appointment counts found for the specified date range.")
    else:
        tk.messagebox.showinfo("Input Error", "Please enter valid start and end dates.")

def fetch_billing_information_gui():
    """GUI to display billing information including patient and staff details."""
    headers, records = fetch_billing_information()
    if headers and records:
        display_table(headers, records, title="Billing Information")
    else:
        tk.messagebox.showinfo("No Data", "No billing information found.")

def fetch_unpaid_billing_records_gui():
    """GUI to display payment status of billing records including patient details."""
    # Prompt the user for payment status
    payment_status = simpledialog.askstring("Payment Status", "Enter payment status (unpaid, payment plan, or paid in full):")
    if payment_status not in ['unpaid', 'payment plan', 'paid in full']:
        messagebox.showerror("Invalid Payment Status", "Please enter a valid payment status.")
        return
    
    headers, records = fetch_unpaid_billing_records(payment_status)
    if headers and records:
        display_table(headers, records, title="Payment Status of Billing Records")
    else:
        messagebox.showinfo("No Data", "No billing records found.")

def fetch_avg_billing_amount_by_illness_gui():
    """GUI to display the average billing amount for each illness description."""
    start_date = simpledialog.askstring("Input", "Enter report start date (YYYY-MM-DD):")
    end_date = simpledialog.askstring("Input", "Enter report end date (YYYY-MM-DD):")
    if start_date and end_date:
        headers, records = fetch_avg_billing_amount_by_illness(start_date, end_date)
        if headers and records:
            display_table(headers, records, title=f"Average Bill by Illness from {start_date} to {end_date}")
        else:
            tk.messagebox.showinfo("No Data", "No appointment counts found for the specified date range.")
    else:
        tk.messagebox.showinfo("Input Error", "Please enter valid start and end dates.")
def main_menu_gui():
    root = tk.Tk()
    root.title("Main Menu")

    # Fetch appointments data button
    tk.Button(root, text="Fetch Appointments Data", command=fetch_appointments_gui).pack(pady=10)

    # Button to fetch and display appointment details
    tk.Button(root, text="Fetch Appointment Details", command=fetch_appointment_details_gui).pack(pady=10)
    
    # Button for fetching patient appointment counts
    tk.Button(root, text="Fetch Patient Appointment Counts", command=fetch_patient_appointment_counts_gui).pack(pady=10)

    # Button for fetching billing information
    tk.Button(root, text="Fetch Billing Information", command=fetch_billing_information_gui).pack(pady=10)
    
    # Button for fetching unpaid billing records
    tk.Button(root, text="Fetch Payment Status Records", command=fetch_unpaid_billing_records_gui).pack(pady=10)
    
    # Button for fetching average billing amount by illness
    tk.Button(root, text="Fetch Average Billing Amount by Illness", command=fetch_avg_billing_amount_by_illness_gui).pack(pady=10)

    # Exit button
    tk.Button(root, text="Exit", command=root.destroy).pack(pady=10)

    root.mainloop()

if __name__ == "__main__":
    main_menu_gui()
